In [40]:
import pandas as pd
import numpy as np
import yfinance as yf

# ========== Input ==========
excel_file = "etf_with_ark_analysis_clean.xlsx"
sheet_name = "Comparable ETFs"
def get_risk_free_rate():
    irx = yf.Ticker("^IRX")
    rate = irx.history(period="5d")["Close"].dropna()
    if not rate.empty:
        return rate.iloc[-1] / 100 / 12
    else:
        raise ValueError("Unable to get ^IRX")

risk_free_rate = get_risk_free_rate()

# ========== Calculate ==========
def compute_max_drawdown(series):
    cummax = series.cummax()
    drawdown = (series - cummax) / cummax
    return drawdown.min()

def compute_return_to_mdd(returns):
    cumulative = (1 + returns).cumprod()
    mdd = compute_max_drawdown(cumulative)
    total_ret = cumulative.iloc[-1] - 1
    return total_ret / abs(mdd) if mdd != 0 else np.nan

def build_df(series_dict):
    df = pd.concat(series_dict, axis=1)
    df.columns = df.columns.get_level_values(0)  
    df.index.name = "Date"
    return df


# ========== Read ETF list ==========
df_etfs = pd.read_excel(excel_file, sheet_name=sheet_name, engine="openpyxl")
tickers = df_etfs.iloc[:, 0].dropna().astype(str).tolist()
print(f"✅ 从Excel读取 {len(tickers)} 个ETF")

# ========== Initial ==========
daily_close_all = {}
sharpe_all = {}
rmd_all = {12: {}, 36: {}, 60: {}}
failed = []

# ========== Main ==========
for ticker in tickers:
    print(f"Handling: {ticker}")
    try:
        data = yf.download(ticker, start="2000-01-01", interval="1d", progress=False)
        close = data["Close"].dropna()
        if close.empty:
            print(f"No data for: {ticker}")
            failed.append(ticker)
            continue

        # Daily close price
        daily_close_all[ticker] = close

        # Monthly return
        monthly_close = close.resample("ME").last()
        returns = monthly_close.pct_change().dropna()

        # Annual Sharpe Ratio
        sharpe = returns.resample("YE").apply(
            lambda r: (r.mean() - risk_free_rate) / r.std() if r.std() > 0 else np.nan
        )
        sharpe_all[ticker] = sharpe

        # Return / Max Drawdown (1Y, 3Y, 5Y)
        for window in [12, 36, 60]:
            if len(returns) >= window:
                rmd = returns.rolling(window).apply(compute_return_to_mdd)
                rmd_all[window][ticker] = rmd
            else:
                rmd_all[window][ticker] = pd.Series(dtype=float)

        print(f"Finished: {ticker}{len(close)} ")

    except Exception as e:
        print(f"Error {ticker}: {e}")
        failed.append(ticker)

# ========== DataFrames ==========
df_close = build_df(daily_close_all)
df_sharpe = build_df(sharpe_all)
df_rmd_1y = build_df(rmd_all[12])
df_rmd_3y = build_df(rmd_all[36])
df_rmd_5y = build_df(rmd_all[60])

# ========== Write Excel ==========
print("\n📤 Writing Excel...")

df_close.reset_index().to_excel("daily_close.xlsx", index=False)
print("daily_close.xlsx Saved")

df_sharpe.reset_index().to_excel("sharpe_ratio.xlsx", index=False)
print("sharpe_ratio.xlsx Saved")

with pd.ExcelWriter("return_mdd.xlsx", engine="openpyxl") as writer:
    df_rmd_1y.reset_index().to_excel(writer, sheet_name="Return-MDD (1Y)", index=False)
    df_rmd_3y.reset_index().to_excel(writer, sheet_name="Return-MDD (3Y)", index=False)
    df_rmd_5y.reset_index().to_excel(writer, sheet_name="Return-MDD (5Y)", index=False)
print("return_mdd.xlsx Saved")

# ========== 错误报告 ==========
if failed:
    print(f"\nThere are {len(failed)} Ticker error:")
    print(", ".join(failed))
else:
    print("\nAll ETFs are successfully processed! ")


✅ 从Excel读取 131 个ETF
Handling: ARKK
Finished: ARKK2615 
Handling: ARKG
Finished: ARKG2615 
Handling: ARKW
Finished: ARKW2638 
Handling: ARKF
Finished: ARKF1545 
Handling: ARKQ
Finished: ARKQ2638 
Handling: ARKX
Finished: ARKX1003 
Handling: ARKB
Finished: ARKB302 
Handling: IZRL
Finished: IZRL1836 
Handling: PRNT
Finished: PRNT2184 
Handling: ILDR
Finished: ILDR963 
Handling: BUYZ
Finished: BUYZ1277 
Handling: FDIF
Finished: FDIF444 
Handling: EVMT
Finished: EVMT732 
Handling: ARKZ
Finished: ARKZ341 
Handling: DYNI
Finished: DYNI341 
Handling: ARKW
Finished: ARKW2638 
Handling: SYNB
Finished: SYNB623 
Handling: TSLT
Finished: TSLT359 
Handling: WGMI
Finished: WGMI785 
Handling: AIYY
Finished: AIYY332 
Handling: SARK
Finished: SARK847 
Handling: TIME
Finished: TIME792 
Handling: MAGS
Finished: MAGS492 
Handling: CEPI
Finished: CEPI76 
Handling: FDG
Finished: FDG1252 
Handling: BMED
Finished: BMED1125 
Handling: MEDX
Finished: MEDX540 
Handling: DRAI
Finished: DRAI177 
Handling: TEK
Finis